# 지도학습-BaseOnly

## 1. 추천 시스템이란?
- 사용자가 좋아할 가능성이 높은 아이템을 예측하는 모델
    - 넷플 영화 추천
    - 유투브 영상 추천
    - 쿠팡 상품 추천


## 2. 준비작업
### 1. 데이터셋 가져오기
#### 1. 분석대상-평점데이터

In [1]:
from hossam import *
from pandas import DataFrame, merge

from surprise import Dataset, Reader, BaselineOnly, accuracy
from surprise.model_selection import train_test_split, GridSearchCV

📦 아이티윌 이광호 강사가 제작한 라이브러리를 사용중입니다.
📚 자세한 사용 방법은 https://py.hossam.kr 을 참고하세요.
📧 Email: leekh4232@gmail.com
🎬 Youtube: https://www.youtube.com/@hossam-codingclub
📝 Blog: https://blog.hossam.kr/
🔖 Version: 0.5.3

✅ 시각화를 위한 한글 글꼴(NotoSansKR-Regular)이 자동 적용되었습니다.


In [2]:
origin = load_data('ml100k-ratings')
print(f"데이터셋 크기: {origin.shape}")
print(f"열 개수: {origin.shape[1]}")
print(f"행 개수: {origin.shape[0]}")
print(origin.info())
origin.head()

943명의 사용자가 1,682편의 영화에 대해 남긴 100,000개의 평점 기록으로 구성된 명시적 평가 기반 추천 시스템 학습용 데이터셋 (출처: University of Minnesota)

컬럼명     의미
---------  ---------
user_id    사용자 ID
item_id    아이템 ID
rating     평점
timestamp  평가 시각

데이터셋 크기: (100000, 4)
열 개수: 4
행 개수: 100000
<class 'pandas.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   user_id    100000 non-null  int64
 1   item_id    100000 non-null  int64
 2   rating     100000 non-null  int64
 3   timestamp  100000 non-null  int64
dtypes: int64(4)
memory usage: 3.1 MB
None


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


#### 2. 분석결과 맵핑 데이터 - 영화 정보


In [3]:
metadata = load_data('ml100k-metadata')
metadata.head()

ml100k-ratings에 포함된 영화 제목, 공개시기, 장르 정보를 담고 있는 데이터 (출처: University of Minnesota)


,item_id,title,release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%20(1995),0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(1995),0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995),0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


## 3. BaselineOnly 모델 적합
### 1. DataFrame을 Dataset 객체로 전환
#### 1. 평점의 범위 확인

In [4]:
rating_min = origin['rating'].min()
rating_max = origin['rating'].max()
print(f'Rating 범위: {rating_min} ~ {rating_max}')

Rating 범위: 1 ~ 5


#### 2. surprise 라이브러리에서 사용할 수 있도록 데이터셋을 변환

In [5]:
# 평점의 범위를 지정하여 Reader 객체 생성
reader = Reader(rating_scale=(rating_min, rating_max))

# Dataset 객체를 생성 - 사용자 식별자, 아이템 식별자, 평점만으로 구성된 데이터 구조가 필요하다.
data = Dataset.load_from_df(origin[['user_id', 'item_id', 'rating']], reader)

data

### 2. BaselineOnly 모델의 주요 하이퍼 파라미터

In [6]:
param_grid = {
    'bsl_options': {
    'method': ['als', 'sgd'],
    'reg': [1, 5, 10, 15, 20],
    'learning_rate': [0.002, 0.005, 0.01],    # sgd일 때만 작용
    'n_epochs': [10, 20, 30]
    }
}

# GridSearchCV 객체를 생성하여 하이퍼파라미터 튜닝을 수행
gs = GridSearchCV(
    BaselineOnly,
    param_grid,
    measures = ['rmse', 'mae'],
    cv = 5,
    n_jobs = -1
)

# GridSearchCV를 사용하여 최적의 하이퍼파라미터 조합을 찾는다,
# -> 원본 데이터 사용
gs.fit(data)

# 최적의 RMSE와 MAE 점수 및 해당 하이퍼파라미터 조합을 출력
print('Best RMSE:', gs.best_score['rmse'])
print('Best Params (RMSE):', gs.best_params['rmse'])

print('Best MAE:', gs.best_score['mae'])
print('Best Params (MAE):', gs.best_params['mae'])

Best RMSE: 0.9442324384970405
Best Params (RMSE): {'bsl_options': {'method': 'als', 'reg': 1, 'learning_rate': 0.002, 'n_epochs': 30}}
Best MAE: 0.7483242786823052
Best Params (MAE): {'bsl_options': {'method': 'als', 'reg': 1, 'learning_rate': 0.002, 'n_epochs': 30}}


## 4. 성능평가
### 1. 훈련, 검증 데이터 분리

In [7]:
# 데이터를 학습용과 테스트용으로 분할 (80% 학습, 20% 테스트)
train_data, test_data = train_test_split(data, test_size = 0.2, random_state = 52)

# 학습용과 테스트용 데이터의 크기를 출력
print(f'Trainset 크기:{train_data.n_ratings}개')
print(f'Testset 크기:{len(test_data)}개')

Trainset 크기:80000개
Testset 크기:20000개


### 2. 최적 모델 재학습

In [8]:
# 최적 파라미터 추출
best_params = gs.best_params['rmse']

# 모델 생성
best_model = BaselineOnly(**best_params)

# 전체 데이터 학습
best_model.fit(train_data)

Estimating biases using als...


### 3. 예측값 생성

In [9]:
predictions = best_model.test(test_data)
predictions[:5]

[Prediction(uid=303, iid=679, r_ui=2.0, est=3.1393275100473677, details={'was_impossible': False}),
 Prediction(uid=308, iid=163, r_ui=4.0, est=3.6883481119038146, details={'was_impossible': False}),
 Prediction(uid=327, iid=663, r_ui=4.0, est=3.5897839643819855, details={'was_impossible': False}),
 Prediction(uid=912, iid=479, r_ui=4.0, est=4.106633675573525, details={'was_impossible': False}),
 Prediction(uid=224, iid=329, r_ui=3.0, est=2.8119332194991884, details={'was_impossible': False})]

### 4. 성능평가 지표 생성

In [10]:
# Train 예측 (trainset 전체를 test 형식으로 변환)
train_predictions = best_model.test(train_data.build_testset())

# Test 예측
test_predictions = best_model.test(test_data)

# 성능 계싼
train_rmse = accuracy.rmse(train_predictions, verbose = False)
train_mae = accuracy.mae(train_predictions, verbose = False)

test_rmse = accuracy.rmse(test_predictions, verbose = False)
test_mae = accuracy.rmse(test_predictions, verbose = False)

# 일반화 오차 차이
rmse_gap = test_rmse - train_rmse
mae_gap = test_mae - train_mae

# 과적합 판정 기준 (RMSE 기준)
# 기준: test RMSE가 train RMSE보다 0.05 이상 크면 과적합 의심
if rmse_gap > 0.05:
    overfit_flag = '과적합 의심'
else:
    overfit_flag = '정상'

# 성능평가표 생성
result_df = DataFrame({
    'Model': ['BaselineOnly'],
    'Train_RMSE': [train_rmse],
    'Test_RMSE': [test_rmse],
    'RMSE_Gap': [rmse_gap],
    'Train_MAE': [train_mae],
    'Test_MAE': [test_mae],
    'MAE_Gap': [mae_gap],
    'Overfitting': [overfit_flag]
})

result_df

,Model,Train_RMSE,Test_RMSE,RMSE_Gap,Train_MAE,Test_MAE,MAE_Gap,Overfitting
0,BaselineOnly,0.922,0.947,0.026,0.731,0.947,0.216,정상


## 5. TopN 추천
### 1. 아직 평가하지 않은 아이템에 대한 예측 수행
#### 1. 예측결과 생성

In [11]:
anti_testset = train_data.build_anti_testset()
predictions = best_model.test(anti_testset)
predictions[:5] # 예측 결과의 일부 출력

[Prediction(uid=234, iid=205, r_ui=3.5317375, est=3.4962269245104007, details={'was_impossible': False}),
 Prediction(uid=234, iid=504, r_ui=3.5317375, est=3.311088596094799, details={'was_impossible': False}),
 Prediction(uid=234, iid=73, r_ui=3.5317375, est=2.9809970216823714, details={'was_impossible': False}),
 Prediction(uid=234, iid=475, r_ui=3.5317375, est=3.3435994193805505, details={'was_impossible': False}),
 Prediction(uid=234, iid=294, r_ui=3.5317375, est=2.6108510945188326, details={'was_impossible': False})]

#### 2. 예측결과 데이터프레임 구성


In [12]:
pred_df = DataFrame(predictions,
                    columns = ['user_id', 'item_id', 'true_rating', 'pred_rating', 'details'])

pred_df.head()

,user_id,item_id,true_rating,pred_rating,details
0,234,205,3.532,3.496,{'was_impossible': False}
1,234,504,3.532,3.311,{'was_impossible': False}
2,234,73,3.532,2.981,{'was_impossible': False}
3,234,475,3.532,3.344,{'was_impossible': False}
4,234,294,3.532,2.611,{'was_impossible': False}


### 2. 특정 사용자에 대한 상위 10개의 추천 영화 검색
#### 1. 44번 사용자에 대한 Top 10 추천 데이터

In [14]:
N = 10
user_id = 44

topn_df = pred_df[pred_df['user_id'] == user_id]

topn_df = (
    topn_df[['user_id', 'item_id', 'pred_rating']]
    .sort_values(['pred_rating'], ascending = [False])
    .groupby('user_id')
    .head(N)
    .reset_index(drop = True)
)

topn_df

,user_id,item_id,pred_rating
0,44,408,4.423
1,44,169,4.385
2,44,483,4.371
3,44,12,4.339
4,44,357,4.268
5,44,114,4.254
6,44,134,4.244
7,44,178,4.238
8,44,657,4.223
9,44,174,4.216


#### 2. 메타데이터와 병합하여 영화 정보 생성

In [16]:
movie_df = topn_df.merge(metadata, on = 'item_id', how = 'left')
movie_df

,user_id,item_id,pred_rating,title,release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,44,408,4.423,"Close Shave, A (1995)",28-Apr-1996,"http://us.imdb.com/M/title-exact?Close%20Shave,%20A%20(1995)",0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
1,44,169,4.385,"Wrong Trousers, The (1993)",01-Jan-1993,"http://us.imdb.com/M/title-exact?Wrong%20Trousers,%20The%20(1993)",0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,44,483,4.371,Casablanca (1942),01-Jan-1942,http://us.imdb.com/M/title-exact?Casablanca%20(1942),0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0
3,44,12,4.339,"Usual Suspects, The (1995)",14-Aug-1995,"http://us.imdb.com/M/title-exact?Usual%20Suspects,%20The%20(1995)",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
4,44,357,4.268,One Flew Over the Cuckoo's Nest (1975),01-Jan-1975,http://us.imdb.com/M/title-exact?One%20Flew%20Over%20the%20Cuckoo's%20Nest%20(1975),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
5,44,114,4.254,Wallace & Gromit: The Best of Aardman Animation (1996),05-Apr-1996,http://us.imdb.com/Title?Wallace+%26+Gromit%3A+The+Best+of+Aardman+Animation+(1996),0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,44,134,4.244,Citizen Kane (1941),01-Jan-1941,http://us.imdb.com/M/title-exact?Citizen%20Kane%20(1941),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
7,44,178,4.238,12 Angry Men (1957),01-Jan-1957,http://us.imdb.com/M/title-exact?12%20Angry%20Men%20(1957),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
8,44,657,4.223,"Manchurian Candidate, The (1962)",01-Jan-1962,"http://us.imdb.com/M/title-exact?Manchurian%20Candidate,%20The%20(1962)",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
9,44,174,4.216,Raiders of the Lost Ark (1981),01-Jan-1981,http://us.imdb.com/M/title-exact?Raiders%20of%20the%20Lost%20Ark%20(1981),0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
